In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [2]:
# imports
from dmagellan.blocker.attrequivalence.attr_equiv_blocker import AttrEquivalenceBlocker
from dmagellan.blocker.overlap.overlapblocker import OverlapBlocker
from dmagellan.blocker.blackbox.blackbox_blocker import BlackBoxBlocker
from dmagellan.blocker.rulebased.rule_based_blocker import RuleBasedBlocker
from dmagellan.feature.autofeaturegen import get_features_for_blocking
from dmagellan.feature.extractfeatures import extract_feature_vecs
from dmagellan.feature.autofeaturegen import get_features_for_matching
from dmagellan.matcher.dtmatcher import DTMatcher
from dmagellan.utils.py_utils.utils import concat_df
from dask import delayed
from dask.threaded import get

In [3]:
import pandas as pd

In [4]:
A = pd.read_csv('citeseer.csv')

In [5]:
B = pd.read_csv('./dblp.csv', low_memory=False)

In [6]:
def profile_table(df, attribute, plot=True):

    unique_values = pd.unique(df[attribute])
    num_missing = sum(pd.isnull(df[attribute]))

    if not plot:
        return  pd.DataFrame({'Property':['Num. Missing Values', 'Num. Unique Values', 'List of Unique Values'],
                               'Value':[num_missing, len(unique_values), sorted(list(unique_values))]})
    else:
        print('Number of unique values: %d\nNumber of missing values: '
              '%d\n\nUnique values:'  % (len(unique_values), num_missing))
#         print(sorted(list(unique_values)))
#         print('\nFrequency plot:\n')
#         d = (pd.DataFrame(df[attribute].value_counts()))
#         ax = sns.barplot(x="index", y=attribute, data=(d).reset_index())
#         ax.set(xlabel=attribute, ylabel='count')
#         ax.grid(b=True, which='major', color='w', linewidth=1.0)
#         ax.set_xticklabels(labels=d.sort_index().index.values, rotation=90)
#         plt.show()

In [6]:
A1 = A[['id', 'title', 'authors']]
B1 = B[['id', 'title', 'authors']]


In [7]:
A2 = A1.dropna()

In [9]:
len(A1), len(A2)

(1823978, 1823846)

In [10]:
A2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823846 entries, 0 to 1823977
Data columns (total 3 columns):
id         int64
title      object
authors    object
dtypes: int64(1), object(2)
memory usage: 55.7+ MB


In [8]:
B2 = B1.dropna()

In [9]:
A2.to_csv('citeseer_nonans.csv', index=False)
B2.to_csv('dblp_nonans.csv', index=False)

In [12]:
B2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2512927 entries, 0 to 2512926
Data columns (total 3 columns):
id         int64
title      object
authors    object
dtypes: int64(1), object(2)
memory usage: 76.7+ MB


In [13]:
A_sample = A2.sample(10000)
B_sample = B2.sample(10000)

In [14]:
# from distributed import Client
# client = Client("128.110.153.177:8786")

In [25]:
x`x

In [15]:
# client.has_what()

In [14]:
# def title_author_sim(ltuple, rtuple):
#     l_title_authors = ltuple['title'] +' '+ ltuple['authors']
#     r_title_authors = rtuple['title'] +' '+ rtuple['authors']    
#     ltokens = l_title_authors.

In [18]:
# # overlap blocker on name
from dask.threaded import get
ob = OverlapBlocker()
C = ob.block_tables(A2.sample(100000), B2.sample(100000), 'id', 'id', 'title', 'title', overlap_size=4, nltable_chunks=2, nrtable_chunks=2, 
                     scheduler=get, compute=True)
len(C)

[########################################] | 100% Completed |  2min 13.3s


827536

In [39]:
import py_stringsimjoin as ssjx

In [40]:
help(ssj.overlap_join)

Help on function overlap_join in module py_stringsimjoin.join.overlap_join:

overlap_join(ltable, rtable, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, comp_op='>=', allow_missing=False, l_out_attrs=None, r_out_attrs=None, l_out_prefix='l_', r_out_prefix='r_', out_sim_score=True, n_jobs=1, show_progress=True)
    Join two tables using overlap measure.
    
    For two sets X and Y, the overlap between them is given by:                       
                                                                                
        :math:`overlap(X, Y) = |X \cap Y|`
    
    Finds tuple pairs from left table and right table such that the overlap 
    between the join attributes satisfies the condition on input threshold. For 
    example, if the comparison operator is '>=', finds tuple pairs whose 
    overlap between the strings that are the values of the join attributes is 
    greater than or equal to the input threshold, as specified in "threshold".
    
    

In [49]:
As = A2.sample(100000)
Bs = B2.sample(100000)

In [43]:
import py_stringmatching as sm
tok = sm.WhitespaceTokenizer()

In [51]:
dd = ssj.overlap_join(As, Bs, 'id', 'id', 'authors', 'authors', tokenizer=tok, threshold=4 )

0%                          100%
[#                             ] | ETA: 00:04:25

KeyboardInterrupt: 

In [48]:
len(dd)

409940

In [21]:
L = pd.read_csv('./matches_citeseer_dblp.csv')

In [23]:
L1 = L.sample(10000)
C1 = C.sample(10000)

In [26]:
L1['label'] = 1
C1['label'] = 0

In [35]:
d = {}
for i in L1.itertuples():
#     print(i)
    x = str(i[1])+'_'+str(i[2])
    d[x] = 0
    


In [33]:
d

{'988550_863052': 0}

In [24]:
dt = DTMatcher(name='dt')

In [25]:
A.head()

,id,title,authors,journal,month,year,publication_type
0,1,An Arithmetic Analogue of Bezouts Theorem,David Mckinnon,NaN,NaN,NaN,NaN
1,2,Thompsons Group F is Not Minimally Almost Convex,"James Belk, Kai-uwe Bux",NaN,NaN,2002.0,NaN
2,3,Cognitive Dimensions Tradeoffs in Tangible User Interface Design,"Darren Edge, Alan Blackwell",NaN,NaN,NaN,NaN
3,4,"ACTIVITY NOUNS, UNACCUSATIVITY, AND ARGUMENT MARKING IN YUKATEKAN SSILA meeting; Special Session...","J. Bohnemeyer, Max Planck, I. Introduction",NaN,NaN,2002.0,NaN
4,5,PS1-6 A6 ULTRASOUND-GUIDED HIFU NEUROLYSIS OF PERIPHERAL NERVES TO TREAT SPASTICITY AND,"J. L. Foley, J. W. Little, F. L. Starr Iii, C. Frantz",NaN,NaN,NaN,NaN


In [ ]:
L = pd.read_csv()